# Lab Assignment One: Exploring Table Data 
**Christopher Cook, Bonita Davis, Anekah Kelley, Davis Lynn**

## 1. Business Understanding 

## 2. Data Understanding

## 3. Data Visualization

## 4. Dimensionality Reduction